In [0]:
from pyspark.sql.functions import col, lit, when
import numpy as np
import pandas as pd

plans = spark.table("workspace.bronze.plans")
users = spark.table("workspace.bronze.users")
silver = spark.table("workspace.silver.subsidy_adjusted_premiums")
subsidy_params = spark.table("workspace.bronze.subsidy_params")

baseline_mc = spark.table("workspace.gold.monte_carlo_risk_metrics")  # baseline only

In [0]:
user = users.first()
user_id = user.user_id
income0 = float(user.annual_income)

In [0]:
plans_demo = plans.filter(col("county") == "Fulton").limit(5)
plan_ids = [r["plan_id"] for r in plans_demo.select("plan_id").collect()]

In [0]:
baseline = baseline_mc.filter(col("user_id") == user_id).filter(col("plan_id").isin(plan_ids))

In [0]:
SCENARIOS = [
    {"scenario_type": "baseline", "income_mult": 1.0, "er_delta": 0, "med_delta": 0.0, "subsidy_expire": False},
    {"scenario_type": "income_plus_10pct", "income_mult": 1.10, "er_delta": 0, "med_delta": 0.0, "subsidy_expire": False},
    {"scenario_type": "add_chronic_med", "income_mult": 1.0, "er_delta": 0, "med_delta": 200.0, "subsidy_expire": False},
    {"scenario_type": "two_er_visits", "income_mult": 1.0, "er_delta": 2, "med_delta": 0.0, "subsidy_expire": False},
    {"scenario_type": "subsidy_expiration", "income_mult": 1.0, "er_delta": 0, "med_delta": 0.0, "subsidy_expire": True},
]

In [0]:
INCOME_STEP = 250
def snap_income(x):
    return int(round(x / INCOME_STEP) * INCOME_STEP)

In [0]:
import pyspark.sql.functions as F

# create scenario income table
scenario_rows = []
for s in SCENARIOS:
    scenario_rows.append((s["scenario_type"], snap_income(income0 * s["income_mult"])))
scenario_income_df = spark.createDataFrame(scenario_rows, ["scenario_type", "income"])

# filter silver to demo plans only, then join on income
silver_demo = silver.filter(col("plan_id").isin(plan_ids))

scenario_prem = scenario_income_df.crossJoin(silver_demo.select("plan_id").distinct()) \
    .join(silver_demo.select("plan_id", "income", "net_premium", "base_premium"),
          on=["plan_id", "income"], how="left")

# apply subsidy expiration override
scenario_prem = scenario_prem.withColumn(
    "net_premium_monthly",
    when(col("scenario_type") == "subsidy_expiration", col("base_premium")).otherwise(col("net_premium"))
).select("scenario_type", "plan_id", "net_premium_monthly")